In [75]:
%%bash

# Dependencies
pip install docx2txt
pip install gensim
pip install keras
pip install nltk
pip install -U scikit-learn
pip install python-docx
pip install tensorflow
pip install nltk
pip install spacy
python3 -m spacy download en_core_web_sm
fi

Requirement already up-to-date: scikit-learn in /home/edward/.local/lib/python3.8/site-packages (1.2.2)


2023-05-17 23:05:33.084677: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-17 23:05:34.995190: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Error while terminating subprocess (pid=7270): 


Defaulting to user installation because normal site-packages is not writeable


In [14]:
# Use if en_core_web_sm not installable via python3 in terminal:
# import spacy
# spacy.cli.download("en_core_web_sm")

In [ ]:
## This code loads the vector file into the word_vectors variable
## Download the vector file from https://fasttext.cc/docs/en/english-vectors.html (first file on the website), unzip the file and store in your local development folder
## Note: This piece of code may take upto an hour or two to run depending on your pc specs.
## My i5 8th gen with 8gig ram took 58mins to run.

from gensim.models import KeyedVectors

# Path to the downloaded .vec file
# path_to_vectors = 'wiki-news-300d-1M.vec'
path_to_vectors = 'wiki.en.vec'
# Load the word vectors
word_vectors = KeyedVectors.load_word2vec_format(path_to_vectors)

# Find similar words
similar_words = word_vectors.most_similar('cat')

# Calculate word similarity
similarity = word_vectors.similarity('cat', 'dog')

# Perform vector arithmetic
result = word_vectors['king'] - word_vectors['man'] + word_vectors['woman']




In [76]:
import spacy

## Function to identify verbs in a sentence
def identify_verbs(sentence):
    # Load the English language model in spaCy
    nlp = spacy.load('en_core_web_sm')
    
    # Process the sentence using spaCy
    doc = nlp(sentence)
    
    # Extract the verbs from the processed sentence
    verbs = [token.lemma_ for token in doc if token.pos_ == 'VERB']
    
    return verbs

In [77]:
## Main piece of code that performs the mapping 

sentences = [
    "apply common data analytics and machine learning algorithms in a big data environment.",
    "use big data streaming technologies.",
    "investigate programming languages"
]

# wiki word vectors no uppercase 
# TODO: Modify this section of the code to use the bloom level verbs from Arragon's spreadsheet, will also need to modify
bloom_levels = ["remembering", "understanding", "applying", "analysing", "evaluating", "creating"]

output_bloom_levels = []

from gensim.models import Word2Vec
from gensim.test.utils import common_texts

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Array of all the PLOs and ULOs (We can couple them together as we're trying to identify Bloom/Solo level here)
lo_sentence_array = []

# TODO: train CLO classification with all data instead of just one course.
for sentence in sentences:
    tokens = nltk.word_tokenize(sentence)
    cleaned_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.lower() not in stop_words]
    lo_sentence_array.append(cleaned_tokens)

# build the vocabulary and train the model
# IMPORTANT, N0TE THAT sg=1 flag specifies Word2Vec to use the Skip Gram Model as designated by the LSTM paper.
model = Word2Vec(sentences=lo_sentence_array,vector_size=100, window=5, min_count=1, workers=4, sg=1)

# train the model with the course's ULOs and PLOs.
model.train([tokens], total_examples=len([tokens]), epochs=10)

lo_sentence_array


# identified_levels = []
final_level = None

for i in range(len(sentences)):
    verbs = identify_verbs(sentences[i])
    # print(verbs)
    score = 0
    for j in range(len(verbs)):
        for k in range(len(bloom_levels)):
            similarity_score = word_vectors.similarity(verbs[j], bloom_levels[k])
            if similarity_score >= score:
                score=similarity_score
                final_level = bloom_levels[k]
                final_level_idx = k
    output_bloom_levels.append(final_level_idx)
    print(sentences[i])
    print("Sentence: ", sentences[i], " Identified blooms level: ", final_level)

print(output_bloom_levels)

### Todos
# Find a way to use skipgrams
# This method only works for blooms since this paper is only based on blooms mapping


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /home/edward/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/edward/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/edward/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


apply common data analytics and machine learning algorithms in a big data environment.
Sentence:  apply common data analytics and machine learning algorithms in a big data environment.  Identified blooms level:  applying
use big data streaming technologies.
Sentence:  use big data streaming technologies.  Identified blooms level:  applying
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
investigate programming languages
Sentence:  investigate programming languages  Identified blooms level:  analysing
[2, 2, 3]


bash: line 13: syntax error near unexpected token `fi'
bash: line 13: `fi'
